<a href="https://colab.research.google.com/github/Hnmani/ML/blob/master/LkoData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DATA FOR LUCKNOW CLASSIFICATION**
Data for this process has to be mined using various resources like website scrapping and Api's

Lucknow:
<br> 
![alt text](https://www.oyorooms.com/blog/wp-content/uploads/2015/11/Screenshot_1.png)

In [0]:
import sqlite3
import requests
import xml.etree.ElementTree as Et
import functools
import pandas as pd
import folium
import matplotlib.cm as cm
from sklearn.cluster import KMeans
import matplotlib.colors as colors

In [0]:
conn=sqlite3.connect('lucknow.sqlite')
cur=conn.cursor()

In [0]:
cur.execute('SELECT name FROM Lucknow')

In [0]:
data=cur.fetchall()

In [0]:
data=data[1:]

In [0]:
#data

In [0]:
apiK=''
appID=''

In [0]:
dat=requests.get("https://geocoder.ls.hereapi.com/6.2/geocode.xml?apiKey="+apiK+"&searchtext=Ismailganj+Lucknow")

In [0]:
tree=Et.fromstring(dat.content)
print([d.text for d in tree.findall('Response/View/Result/Location/Address/Label')])

['Ismailganj, Chinhat, Lucknow, UP, India']


In [0]:
Lat = [d.text for d in tree.findall('Response/View/Result/Location/DisplayPosition/Latitude')][0]
Long = [d.text for d in tree.findall('Response/View/Result/Location/DisplayPosition/Longitude')][0]
print(Lat,Long)

26.87528 81.00891


In [0]:
Val=functools.reduce(lambda a,b: a+'+'+b,data[0][0].split())

In [0]:
def get_dat(c):
  Val=functools.reduce(lambda a,b: a+'+'+b , c.split())
  dat=requests.get('https://geocoder.ls.hereapi.com/6.2/geocode.xml?apiKey='+apiK+'&searchtext='+Val)
  tree=Et.fromstring(dat.content)
  lt = [d.text for d in tree.findall('Response/View/Result/Location/DisplayPosition/Latitude')]
  ln = [d.text for d in tree.findall('Response/View/Result/Location/DisplayPosition/Longitude')]
  nm = [d.text for d in tree.findall('Response/View/Result/Location/Address/Label')]
  return lt,ln,nm

In [0]:
#locations=[get_dat(c[0]) for c in data]


In [0]:
locs_to_compare=list()
for a,b,nm in locations:
  if len(a)>0 and len(b)>0:
    if float(a[0])>27 or float(a[0])<26 or float(b[0])>82 or float(b[0])<80:
      continue
    locs_to_compare.append([a[0],b[0],nm[0]])

#locs_to_compare

In [0]:
locs=pd.DataFrame(locs_to_compare)

In [0]:
locs=locs.rename(columns={0:'Latitude',1:'Longitude',2:'Name'})

In [0]:

locs['ID']=locs.index
locs['ID']+=1
locs.head()
locs.to_csv('Locs.csv')

# Lets See Our Locations On A MAP

In [0]:
Lko=get_dat('Lucknow India')


Lucknow India
['26.85472'] ['80.92135']


'26.85472'

In [0]:

Map=folium.Map(location=[float(Lko[0][0]),float(Lko[1][0])],zoom_start=10)
for lat,lang,name in locs_to_compare:
  #print(lat,lang)
  label='{}'.format(name)
  label=folium.Popup(label,parse_html=True)
  folium.CircleMarker(
      [float(lat),float(lang)],
      radius=10,
      color='blue',
      popup=label,
      fill=True,
      fill_color='#3186cc',
      fill_opacity=0.7,
      parse_html=False
  ).add_to(Map)
Map

Using Google Places Api Now the nearby shops and stores are to be explored

In [0]:
google_APIK=""

In [0]:
locs_to_compare[0]

['26.79247', '80.88419']

In [0]:
# dt=[]
# for a,b in locs_to_compare:
#   d_temp=requests.get('https://maps.googleapis.com/maps/api/place/nearbysearch/xml?location='+a+','+b+'&radius=1000&key='+google_APIK)
#   dt.append(d_temp)

In [0]:
pert=dt.copy()

In [0]:
dataf=[]
K=0
for name,l in zip(pert,locs_to_compare):
  K+=1
  temp=[]
  t=Et.fromstring(name.content)
  res=t.findall('result/')
  Lt=t.findall('result/geometry/location/lat')
  LO=t.findall('result/geometry/location/lng')
  k=0
  for i in res:
    if i.tag=='name':
      temp_long=Lt[k].text
      temp_lat=LO[k].text
      temp_name=i.text
      k+=1
    if i.tag=='vicinity':
      temp_add=i.text
    if i.tag=='type':
      if temp_name=='Lucknow':
        continue
      temp.append((temp_name,temp_add,i.text,l[0],l[1],temp_long,temp_lat,K))
  dataf.append(temp)

In [0]:
DF=pd.DataFrame([item for k in dataf for item in k])

In [0]:
DF.rename(columns={0:'Name',1:'Address',2:'Type',3:'Latitude_MAIN',4:'Longitude_MAIN',5:'Latitude_temp',6:'Longitude_temp',7:'ID'}, inplace=True)

In [0]:
DF.head()

,Name,Address,Type,Latitude_MAIN,Longitude_MAIN,Latitude_temp,Longitude_temp,ID
0,Bank of India,"Star House, Near Bank Pul (Near (ROB)), Gomti ...",bank,26.79247,80.88419,26.7897400,80.8890300,1
1,Bank of India,"Star House, Near Bank Pul (Near (ROB)), Gomti ...",finance,26.79247,80.88419,26.7897400,80.8890300,1
2,Bank of India,"Star House, Near Bank Pul (Near (ROB)), Gomti ...",point_of_interest,26.79247,80.88419,26.7897400,80.8890300,1
3,Bank of India,"Star House, Near Bank Pul (Near (ROB)), Gomti ...",establishment,26.79247,80.88419,26.7897400,80.8890300,1
4,State Bank,"Balaji Complex, Kanpur Road, Kanpur Road, Kris...",atm,26.79247,80.88419,26.7956800,80.8927600,1


In [0]:
DF.shape

(4351, 8)

In [0]:
#DF.to_csv('LKO.csv')

In [0]:
DF.groupby('ID').count()

,Name,Address,Type,Latitude_MAIN,Longitude_MAIN,Latitude_temp,Longitude_temp
ID,,,,,,,
1,51,51,51,51,51,51,51
2,57,57,57,57,57,57,57
3,62,62,62,62,62,62,62
4,53,53,53,53,53,53,53
5,53,53,53,53,53,53,53
...,...,...,...,...,...,...,...
70,64,64,64,64,64,64,64
71,62,62,62,62,62,62,62
72,58,58,58,58,58,58,58


In [0]:
print('There are {} different Types of Shop '.format(len(DF['Type'].unique())))

There are 70 different Types of Shop 


In [0]:
##Doing One Hot Encoding

Lko_onehot= pd.get_dummies(DF[['Type']],prefix='',prefix_sep='')
Lko_onehot['ID']=DF['ID']
#To Move the ID COLUMN IN FRONT 
cols=[Lko_onehot.columns[-1]]+list(Lko_onehot.columns[:-1])
Lko_onehot=Lko_onehot[cols]

Lko_onehot.head()

,ID,accounting,atm,bakery,bank,bar,beauty_salon,bicycle_store,book_store,cafe,car_dealer,car_rental,car_repair,church,clothing_store,courthouse,dentist,doctor,electrician,electronics_store,establishment,finance,food,furniture_store,gas_station,general_contractor,grocery_or_supermarket,gym,hair_care,health,hindu_temple,home_goods_store,hospital,insurance_agency,jewelry_store,laundry,lawyer,liquor_store,local_government_office,locality,lodging,mosque,movie_theater,moving_company,museum,park,pharmacy,physiotherapist,place_of_worship,point_of_interest,police,political,post_office,real_estate_agency,restaurant,roofing_contractor,school,secondary_school,shoe_store,shopping_mall,storage,store,sublocality,sublocality_level_1,sublocality_level_2,sublocality_level_3,supermarket,tourist_attraction,travel_agency,university,veterinary_care
0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
Lko_onehot.shape

(4351, 71)

In [0]:
GROUPPED_DATA = Lko_onehot.groupby('ID').mean().reset_index()

In [0]:
GROUPPED_DATA.head()

,ID,accounting,atm,bakery,bank,bar,beauty_salon,bicycle_store,book_store,cafe,car_dealer,car_rental,car_repair,church,clothing_store,courthouse,dentist,doctor,electrician,electronics_store,establishment,finance,food,furniture_store,gas_station,general_contractor,grocery_or_supermarket,gym,hair_care,health,hindu_temple,home_goods_store,hospital,insurance_agency,jewelry_store,laundry,lawyer,liquor_store,local_government_office,locality,lodging,mosque,movie_theater,moving_company,museum,park,pharmacy,physiotherapist,place_of_worship,point_of_interest,police,political,post_office,real_estate_agency,restaurant,roofing_contractor,school,secondary_school,shoe_store,shopping_mall,storage,store,sublocality,sublocality_level_1,sublocality_level_2,sublocality_level_3,supermarket,tourist_attraction,travel_agency,university,veterinary_care
0,1,0.000000,0.019608,0.000000,0.019608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.019608,0.000000,0.0,0.000000,0.352941,0.039216,0.019608,0.0,0.0,0.000000,0.0,0.0,0.0,0.039216,0.000000,0.0,0.000000,0.019608,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.352941,0.0,0.019608,0.000000,0.000000,0.000000,0.0,0.039216,0.000000,0.0,0.0,0.0,0.019608,0.019608,0.019608,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0
1,2,0.000000,0.017544,0.035088,0.017544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.315789,0.017544,0.035088,0.0,0.0,0.017544,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.017544,0.0,0.0,0.000000,0.315789,0.0,0.017544,0.000000,0.017544,0.000000,0.0,0.052632,0.000000,0.0,0.0,0.0,0.070175,0.017544,0.017544,0.000000,0.0,0.0,0.017544,0.0,0.000000,0.0
2,3,0.000000,0.048387,0.000000,0.016129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.032258,0.290323,0.048387,0.048387,0.0,0.0,0.016129,0.0,0.0,0.0,0.000000,0.016129,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.016129,0.290323,0.0,0.016129,0.000000,0.000000,0.016129,0.0,0.016129,0.016129,0.0,0.0,0.0,0.064516,0.016129,0.016129,0.000000,0.0,0.0,0.000000,0.0,0.016129,0.0
3,4,0.000000,0.018868,0.000000,0.018868,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.018868,0.0,0.000000,0.000000,0.0,0.000000,0.339623,0.018868,0.018868,0.0,0.0,0.037736,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.018868,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.339623,0.0,0.018868,0.000000,0.018868,0.000000,0.0,0.056604,0.000000,0.0,0.0,0.0,0.018868,0.018868,0.018868,0.000000,0.0,0.0,0.000000,0.0,0.018868,0.0
4,5,0.018868,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.018868,0.0,0.000000,0.018868,0.0,0.000000,0.339623,0.056604,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.037736,0.000000,0.0,0.018868,0.018868,0.0,0.0,0.018868,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.339623,0.0,0.018868,0.018868,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.037736,0.018868,0.000000,0.018868,0.0,0.0,0.000000,0.0,0.000000,0.0


In [0]:
GROUPPED_DATA.shape

(74, 71)

In [0]:
import numpy as np

def retMCV(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

no_top_ven=10
indicators=['st','nd','rd']
cols=['ID']
for ind in np.arange(no_top_ven):
  try:
    cols.append('{}{} Most Common Venue'.format(ind+1,indicators[ind]))
  except:
    cols.append('{}{} Most Common Venue'.format(ind+1,'th'))
Lksort=pd.DataFrame(columns=cols)
Lksort['ID']=GROUPPED_DATA['ID']
for ind in np.arange(GROUPPED_DATA.shape[0]):
  Lksort.iloc[ind,1:]=retMCV(GROUPPED_DATA.iloc[ind,:],no_top_ven)

In [0]:
Lksort.head()

,ID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,establishment,point_of_interest,health,finance,school,political,store,food,dentist,insurance_agency
1,2,point_of_interest,establishment,store,school,bakery,food,political,park,finance,real_estate_agency
2,3,establishment,point_of_interest,store,finance,food,atm,electronics_store,bank,place_of_worship,sublocality_level_1
3,4,point_of_interest,establishment,school,general_contractor,political,store,lodging,finance,clothing_store,sublocality
4,5,establishment,point_of_interest,finance,health,store,accounting,clothing_store,insurance_agency,lawyer,doctor


In [0]:
kClus=5
LKOCLUS=GROUPPED_DATA.drop('ID',1)
kmeans=KMeans(n_clusters=kClus,random_state=0).fit(LKOCLUS)
kmeans.labels_
Lksort.insert(0,'Cluster Labels',kmeans.labels_)

In [0]:

Lko_merged= Lksort.copy()
Lko_merged=Lko_merged.join(locs.set_index('ID'), on ='ID')
Lko_merged.head()

,Cluster Labels,ID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude,Name
0,2,1,establishment,point_of_interest,health,finance,school,political,store,food,dentist,insurance_agency,26.79247,80.88419,"Ambedkar Nagar, Krishna Nagar, Lucknow, UP, India"
1,2,2,point_of_interest,establishment,store,school,bakery,food,political,park,finance,real_estate_agency,26.76025,80.92008,"1, Omax City, Sarojini Nagar, Lucknow 226301, ..."
2,1,3,establishment,point_of_interest,store,finance,food,atm,electronics_store,bank,place_of_worship,sublocality_level_1,26.7878,80.9672,"Ibrahimpur, Lucknow, UP, India"
3,2,4,point_of_interest,establishment,school,general_contractor,political,store,lodging,finance,clothing_store,sublocality,26.75589,80.92925,"Sharda Nagar, Aurangabad Khalsa, Lucknow, UP, ..."
4,2,5,establishment,point_of_interest,finance,health,store,accounting,clothing_store,insurance_agency,lawyer,doctor,26.84236,80.87682,"Shaheed Bhagat Singh Marg, Rajaji Puram-Block ..."


In [0]:

Map=folium.Map(location=[float(Lko[0][0]),float(Lko[1][0])],zoom_start=10)

x = np.arange(kClus)
ys = [i + x + (i*x)**2 for i in range(kClus)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]



for lat,lon,c,nm in zip(Lko_merged['Latitude'],Lko_merged['Longitude'],Lko_merged['Cluster Labels'],Lko_merged['Name']):
  #print(lat,lang)
  i+=1
  label='{} Cluster={}'.format(nm,c)
  label=folium.Popup(label,parse_html=True)
  folium.CircleMarker(
      [float(lat),float(lon)],
      radius=10,
      color=rainbow[c-1],
      popup=label,
      fill=True,
      fill_color=rainbow[c-1],
      fill_opacity=0.7,
      parse_html=False
  ).add_to(Map)
Map

In [0]:
Lko_merged.loc[Lko_merged['Cluster Labels'] == 0, Lko_merged.columns[[1] + list(range(2, Lko_merged.shape[1]))]]

,ID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
10,11,establishment,point_of_interest,store,home_goods_store,health,sublocality_level_3,bank,jewelry_store,sublocality,travel_agency,26.85643,80.89118
17,18,establishment,point_of_interest,store,finance,travel_agency,food,bank,grocery_or_supermarket,electronics_store,cafe,26.86538,80.95977
23,24,establishment,point_of_interest,store,car_dealer,bank,finance,food,local_government_office,place_of_worship,car_repair,26.87148,80.92736
28,29,point_of_interest,establishment,store,clothing_store,lodging,political,food,health,place_of_worship,electronics_store,26.94016,81.20876
32,33,establishment,point_of_interest,store,health,atm,gym,lodging,food,finance,school,26.81038,80.89383
36,37,point_of_interest,establishment,store,clothing_store,lodging,political,food,health,place_of_worship,electronics_store,26.94016,81.20876
45,46,establishment,point_of_interest,store,food,finance,bakery,grocery_or_supermarket,lodging,electronics_store,doctor,26.8762,80.95586
46,47,point_of_interest,establishment,store,finance,food,pharmacy,home_goods_store,furniture_store,political,sublocality,26.84811,80.90487
48,49,point_of_interest,establishment,store,clothing_store,lodging,political,food,health,place_of_worship,electronics_store,26.94016,81.20876
50,51,point_of_interest,establishment,store,clothing_store,lodging,political,food,health,place_of_worship,electronics_store,26.94016,81.20876


In [0]:
Lko_merged.loc[Lko_merged['Cluster Labels'] == 1, Lko_merged.columns[[1] + list(range(2, Lko_merged.shape[1]))]]

,ID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
2,3,establishment,point_of_interest,store,finance,food,atm,electronics_store,bank,place_of_worship,sublocality_level_1,26.7878,80.9672
11,12,establishment,point_of_interest,store,finance,food,atm,electronics_store,bank,place_of_worship,sublocality_level_1,26.7878,80.9672
13,14,point_of_interest,establishment,finance,atm,bank,store,political,gas_station,home_goods_store,food,26.84018,80.90054
16,17,point_of_interest,establishment,finance,bank,store,place_of_worship,restaurant,atm,food,school,26.79735,80.87771
21,22,establishment,point_of_interest,store,finance,atm,bank,food,furniture_store,place_of_worship,health,26.78635,80.94477
24,25,point_of_interest,establishment,bank,finance,atm,health,store,sublocality_level_3,beauty_salon,sublocality,26.84832,80.91464
26,27,establishment,point_of_interest,finance,lodging,store,bank,grocery_or_supermarket,travel_agency,atm,food,26.86725,80.95282
31,32,establishment,point_of_interest,store,bank,finance,health,clothing_store,lawyer,school,doctor,26.85797,80.92407
34,35,establishment,point_of_interest,finance,bank,store,university,atm,car_dealer,lodging,courthouse,26.87528,81.00891
35,36,establishment,point_of_interest,finance,bank,store,university,atm,car_dealer,lodging,courthouse,26.87528,81.00891


In [0]:
Lko_merged.loc[Lko_merged['Cluster Labels'] == 2, Lko_merged.columns[[1] + list(range(2, Lko_merged.shape[1]))]]

,ID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
0,1,establishment,point_of_interest,health,finance,school,political,store,food,dentist,insurance_agency,26.79247,80.88419
1,2,point_of_interest,establishment,store,school,bakery,food,political,park,finance,real_estate_agency,26.76025,80.92008
3,4,point_of_interest,establishment,school,general_contractor,political,store,lodging,finance,clothing_store,sublocality,26.75589,80.92925
4,5,establishment,point_of_interest,finance,health,store,accounting,clothing_store,insurance_agency,lawyer,doctor,26.84236,80.87682
6,7,point_of_interest,establishment,store,school,bakery,food,political,park,finance,real_estate_agency,26.7602,80.91997
8,9,establishment,point_of_interest,store,general_contractor,school,electronics_store,health,political,finance,lodging,26.85856,80.87309
9,10,point_of_interest,establishment,school,general_contractor,political,store,lodging,finance,clothing_store,sublocality,26.75589,80.92925
12,13,point_of_interest,establishment,store,school,home_goods_store,insurance_agency,furniture_store,moving_company,pharmacy,clothing_store,26.87485,80.88768
14,15,point_of_interest,establishment,hospital,store,health,bank,school,finance,jewelry_store,secondary_school,26.84241,80.86114
18,19,point_of_interest,establishment,store,food,finance,political,atm,health,bank,university,26.8338,80.95762


In [0]:
Lko_merged.loc[Lko_merged['Cluster Labels'] == 3, Lko_merged.columns[[1] + list(range(2, Lko_merged.shape[1]))]]

,ID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
7,8,establishment,point_of_interest,finance,store,storage,car_repair,food,political,lodging,general_contractor,26.8398,80.90366
20,21,point_of_interest,establishment,store,lodging,political,hindu_temple,home_goods_store,jewelry_store,university,food,26.82085,80.91199
25,26,point_of_interest,establishment,health,lodging,school,food,museum,political,real_estate_agency,finance,26.86981,81.03606
33,34,establishment,point_of_interest,finance,atm,political,locality,real_estate_agency,restaurant,food,lodging,26.89124,81.05794
37,38,establishment,point_of_interest,store,local_government_office,lodging,finance,accounting,school,university,place_of_worship,26.83208,80.94082
39,40,point_of_interest,establishment,health,store,electronics_store,lodging,home_goods_store,insurance_agency,sublocality_level_2,political,26.856,80.95912
42,43,point_of_interest,establishment,university,lodging,store,local_government_office,bank,finance,car_dealer,school,26.86839,80.9332
56,57,establishment,point_of_interest,health,store,lodging,political,car_repair,finance,dentist,clothing_store,26.88669,80.99334
62,63,establishment,point_of_interest,lodging,political,police,health,lawyer,finance,hospital,atm,26.87418,80.94462
63,64,point_of_interest,establishment,store,lodging,local_government_office,bank,finance,car_dealer,car_repair,political,26.87182,80.93111


In [0]:
Lko_merged.loc[Lko_merged['Cluster Labels'] == 4, Lko_merged.columns[[1] + list(range(2, Lko_merged.shape[1]))]]

,ID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
5,6,establishment,point_of_interest,lodging,bank,store,finance,car_repair,political,atm,clothing_store,26.83887,80.9449
15,16,establishment,point_of_interest,lodging,store,food,restaurant,finance,bank,insurance_agency,health,26.84841,80.95065
27,28,point_of_interest,establishment,lodging,store,food,clothing_store,restaurant,health,grocery_or_supermarket,political,26.853,80.9368
29,30,establishment,point_of_interest,lodging,store,finance,clothing_store,political,restaurant,atm,insurance_agency,26.85178,80.93387
41,42,point_of_interest,establishment,lodging,store,political,car_repair,car_rental,general_contractor,bicycle_store,sublocality,26.84923,80.99726
44,45,establishment,point_of_interest,lodging,store,food,clothing_store,finance,restaurant,school,shoe_store,26.84837,80.94386
49,50,establishment,point_of_interest,lodging,store,travel_agency,home_goods_store,movie_theater,health,political,car_repair,26.86375,81.00033
57,58,point_of_interest,establishment,lodging,sublocality,finance,clothing_store,store,political,sublocality_level_2,bank,26.84216,80.92879


In [0]:
Lko_merged.to_csv('FINAL_LKO.csv')